In [3]:
#Demonstration
import math
    
def decoratorlog(f):
    def wrapper(*args):
        result =  f(*args)
        print(f"{f.__name__} de {args} est: {result}") 
        return result
    return wrapper

In [24]:
def createDataSet(*args):
    # A People with Age 0 -> young 1 -> old, Income low -> 0 high -> 1
    dataSet = [[0, 0, 'no buy'],  # young, low income
               [1, 1, 'buy'],    # old, high income
               [0, 1, 'buy'],    # young, high income
               [1, 1, 'buy'],    # old, high income
               [0, 0, 'no buy'], # young, low income
               [1, 1, 'buy'],    # old, high income
               [1, 0, 'buy'], # old, low income
               [0, 0, 'no buy'], # young, low income
               [0, 0, 'no buy'], # young, low income
               [1, 1, 'buy']]    # old, high income
    
    # Labels represent the feature names for Age and Income
    labels = ['Age', 'Income']
    
    return dataSet, labels

dataSet, labels = createDataSet()

# Output the new dataset
print("Dataset:", dataSet)
print("Labels:", labels)


Dataset: [[0, 0, 'no buy'], [1, 1, 'buy'], [0, 1, 'buy'], [1, 1, 'buy'], [0, 0, 'no buy'], [1, 1, 'buy'], [1, 0, 'buy'], [0, 0, 'no buy'], [0, 0, 'no buy'], [1, 1, 'buy']]
Labels: ['Age', 'Income']


In [25]:
from math import log
#@decoratorlog
def calcShannonEnt(dataSet):
     numEntries = len(dataSet)
     labelCounts = {}
     for featVec in dataSet: 
         currentLabel = featVec[-1] 
         if currentLabel not in labelCounts.keys():
             labelCounts[currentLabel] = 0 
         labelCounts[currentLabel] += 1 
     shannonEnt = 0.0
     for key in labelCounts:
         prob = float(labelCounts[key])/numEntries
         shannonEnt -= prob * log(prob,2) 
     return shannonEnt
    
calcShannonEnt(dataSet)

0.9709505944546686

In [26]:
#Splitting Dataset
#@decoratorlog
def splitDataSet(dataSet, axis, value):
    retDataset =[]
    for featVec in dataSet:
        if featVec[axis] == value:
            reducedFeatVec = featVec[:axis]
            reducedFeatVec.extend(featVec[axis+1:])
            retDataset.append(reducedFeatVec)
    return retDataset
dataSet, labels = createDataSet()
splitDataSet(dataSet, 0, 1)
splitDataSet(dataSet, 0, 0)
splitDataSet(dataSet, 1, 0)
splitDataSet(dataSet, 1, 1)

[[1, 'buy'], [0, 'buy'], [1, 'buy'], [1, 'buy'], [1, 'buy']]

In [27]:
dataSet, labels = createDataSet()
print(dataSet[0])
print(labels)
@decoratorlog
def chooseBestFeatureToSplit(dataSet):
    numFeatures = len(dataSet[0]) - 1
    baseEntropy = calcShannonEnt(dataSet)
    bestInfoGain = 0.0
    bestFeature = -1
    for i in range(numFeatures):
        featList = [example[i] for example in dataSet]
        uniqueVals = set(featList) 
        newEntropy = 0.0
        for value in uniqueVals:
            subDataset = splitDataSet(dataSet, i, value)
            prob = len(subDataset) / float(len(dataSet))
            newEntropy += prob * calcShannonEnt(subDataset)
        infoGain = baseEntropy - newEntropy 
        if (infoGain > bestInfoGain):
            bestInfoGain = infoGain 
            bestFeature = i 
    return bestFeature 

chooseBestFeatureToSplit(dataSet)   
    
    

[0, 0, 'no buy']
['Age', 'Income']
chooseBestFeatureToSplit de ([[0, 0, 'no buy'], [1, 1, 'buy'], [0, 1, 'buy'], [1, 1, 'buy'], [0, 0, 'no buy'], [1, 1, 'buy'], [1, 0, 'buy'], [0, 0, 'no buy'], [0, 0, 'no buy'], [1, 1, 'buy']],) est: 0


0

In [28]:
import operator
#@decoratorlog
def majorityCnt(classList):
    classCount={}
    for vote in classList:
        if vote not in classCount.keys():
            classCount[vote] = 0
        classCount[vote] += 1
    sortedClassCount = sorted(classCount.items(), key=operator.itemgetter(1), reverse=True)
    return sortedClassCount[0][0]
    
majorityCnt(['yes', 'yes', 'no'])
majorityCnt(['yes', 'yes', 'no','no','maybe'])
majorityCnt(['yes', 'yes', 'no','no','no'])

'no'

In [29]:
#@decoratorlog
def createTree(dataSet, labels):
     classList = [example[-1] for example in dataSet]
     if classList.count(classList[0]) == len(classList):
         return classList[0] 
     if len(dataSet[0]) == 1: 
         return majorityCnt(classList) 
     bestFeat = chooseBestFeatureToSplit(dataSet)
     bestFeatLabel = labels[bestFeat]
     myTree = {bestFeatLabel:{}}
     del(labels[bestFeat]) 
     featValues = [example[bestFeat] for example in dataSet] 
     uniqueVals = set(featValues) 
     for value in uniqueVals:
         subLabels = labels[:] 
         myTree[bestFeatLabel][value] = createTree(splitDataSet(dataSet, bestFeat, value),subLabels)
     return myTree
dataSet, labels = createDataSet()
createTree(dataSet, labels)


chooseBestFeatureToSplit de ([[0, 0, 'no buy'], [1, 1, 'buy'], [0, 1, 'buy'], [1, 1, 'buy'], [0, 0, 'no buy'], [1, 1, 'buy'], [1, 0, 'buy'], [0, 0, 'no buy'], [0, 0, 'no buy'], [1, 1, 'buy']],) est: 0
chooseBestFeatureToSplit de ([[0, 'no buy'], [1, 'buy'], [0, 'no buy'], [0, 'no buy'], [0, 'no buy']],) est: 0


{'Age': {0: {'Income': {0: 'no buy', 1: 'buy'}}, 1: 'buy'}}

In [31]:
dataSet, labels = createDataSet()
label_input = labels[:]
tree = createTree(dataSet, label_input)

#@decoratorlog
def classify(inputTree, featLabels, testVec):
    firstStr = list(inputTree.keys())[0]
    secondDict = inputTree[firstStr]
    featIndex = featLabels.index(firstStr) 
    for key in secondDict.keys():
        if testVec[featIndex] == key:
            if type(secondDict[key]).__name__=='dict':
                classLabel = classify(secondDict[key],featLabels,testVec)
            else: classLabel = secondDict[key]
    return classLabel
classify(tree, labels, [1, 0])

chooseBestFeatureToSplit de ([[0, 0, 'no buy'], [1, 1, 'buy'], [0, 1, 'buy'], [1, 1, 'buy'], [0, 0, 'no buy'], [1, 1, 'buy'], [1, 0, 'buy'], [0, 0, 'no buy'], [0, 0, 'no buy'], [1, 1, 'buy']],) est: 0
chooseBestFeatureToSplit de ([[0, 'no buy'], [1, 'buy'], [0, 'no buy'], [0, 'no buy'], [0, 'no buy']],) est: 0


'buy'